In [1]:
from rich import print
import os
from openai import OpenAI
from datasets import Dataset, DatasetDict, load_dataset
import json
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

True

In [2]:
client = OpenAI(
    base_url="https://integrate.api.nvidia.com/v1",
    api_key=os.environ["NVIDIA_API_KEY"]
)

In [3]:
messages = [
    {
        "role": "user",
        "content": "What is the capital of Iran?"
    },
    {
        "role": "assistant",
        "content": "The capital of Iran is Tehran."
    },
]

response = client.chat.completions.create(
    model="nvidia/nemotron-4-340b-reward",
    messages=messages,
)

print(response.choices[0].logprobs.content)

[
    ChatCompletionTokenLogprob(token='helpfulness', bytes=None, logprob=4.15625, top_logprobs=[]),
    ChatCompletionTokenLogprob(token='correctness', bytes=None, logprob=4.125, top_logprobs=[]),
    ChatCompletionTokenLogprob(token='coherence', bytes=None, logprob=4.09375, top_logprobs=[]),
    ChatCompletionTokenLogprob(token='complexity', bytes=None, logprob=0.55078125, top_logprobs=[]),
    ChatCompletionTokenLogprob(token='verbosity', bytes=None, logprob=0.52734375, top_logprobs=[])
]

In [4]:
def get_scores_from_response(score_response_template):
    logprobs = score_response_template.choices[0].logprobs.content
    score_dict = {}
    for score in logprobs:
        score_dict[score.token] = score.logprob
    return score_dict

print(get_scores_from_response(response))

{
    'helpfulness': 4.15625,
    'correctness': 4.125,
    'coherence': 4.09375,
    'complexity': 0.55078125,
    'verbosity': 0.52734375
}

In [5]:
def get_response_and_scores(client, model, question, response_content):
    messages = [
        {
            "role": "user",
            "content": question
        },
        {
            "role": "assistant",
            "content": response_content
        },
    ]

    response = client.chat.completions.create(
        model=model,
        messages=messages,
    )

    scores = get_scores_from_response(response)
    return scores

In [64]:
import json

def read_synthetic_data(file_path):
    data_list = []
    with open(file_path, 'r') as file:
        for line in file:
            entry = json.loads(line)
            one_line_dict = {}
            one_line_dict["instruction"] = entry['instruction']
            one_line_dict["response"] = entry['responses']
            data_list.append(one_line_dict)
    return data_list

synthetic_data = read_synthetic_data("synthetic_data.jsonl")
len(synthetic_data)

335

In [7]:
synthetic_data[0]

{'instruction': 'Create a new branch for my feature',
 'response': "To create a new branch for your feature, use the following Git command:\n\n`git branch feature/your-feature-name`\n\nReplace `your-feature-name` with a descriptive name for your feature. This will create a new branch, but you'll still be on the current branch.\n\nTo switch to the new branch, use:\n\n`git checkout feature/your-feature-name`\n\nAlternatively, you can create and switch to the new branch in one step with:\n\n`git checkout -b feature/your-feature-name`"}

In [8]:
def process_instruction_response_score(client, model, synthetic_data):
    score_list = []
    for instruction_response_pair in synthetic_data:
        instruction = instruction_response_pair["instruction"]
        response = instruction_response_pair["response"]
        score = get_response_and_scores(client, model, instruction, response)
        score_list.append(score)
    return score_list

In [9]:
score_list = process_instruction_response_score(client, "nvidia/nemotron-4-340b-reward", synthetic_data)

In [65]:
helpfulness_THRESHOLD = 3
verbosity_THRESHOLD = 2.5
synthetic_data = [data for i, data in enumerate(synthetic_data) 
                  if not (score_list[i]["helpfulness"] < helpfulness_THRESHOLD or 
                          score_list[i]["verbosity"] > verbosity_THRESHOLD)]
len(synthetic_data)

322

In [67]:
with open('synthetic_data_filtered.jsonl', 'w') as f:
    for item in synthetic_data:
        f.write(json.dumps(item))
        f.write('\n')

In [71]:
from huggingface_hub import login
login()

In [72]:
with open(f'synthetic_data_filtered.jsonl', 'r') as f:
    data = [json.loads(line) for line in f]
dataset = Dataset.from_list(data)
dataset_dict = DatasetDict({"train": dataset})
dataset_dict.push_to_hub("hesamsheikh/git-prompt")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/hesamsheikh/git-prompt/commit/b84898544f874d323812feed0abef8093d5e0aa8', commit_message='Upload dataset', commit_description='', oid='b84898544f874d323812feed0abef8093d5e0aa8', pr_url=None, pr_revision=None, pr_num=None)